# Study cityjet scraping
## Install
1) Chromedriver

2) > pip install splinter pandas

In [141]:
from splinter import Browser
from selenium import webdriver
import pandas as pd
from datetime import datetime as DateTime, timedelta as TimeDelta
import time
import random
import os

# params

flight_from = 'London'
flight_to = 'Amsterdam'
date_from = (DateTime.today() + TimeDelta(days=17)).strftime('%d/%m/%Y')
date_to = (DateTime.today() + TimeDelta(days=25)).strftime('%d/%m/%Y')
delayed = 0
scrapedCsv = "Study_scraped_file.csv"

print(flight_from, flight_to, date_from, date_to)

# clean targeted csv
if os.path.isfile(scrapedCsv):
    os.remove(scrapedCsv)


London Amsterdam 12/07/2018 20/07/2018


In [142]:
# open a browser

# From http://www.xroxy.com/
#PROXY = "66.70.147.196:3128"
#PROXY = "178.32.181.66:3128"
PROXY = "142.4.209.32:3128"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

browser = Browser('chrome', options=chrome_options)
browser.driver.set_window_size(800, 600)

In [143]:
browser.visit('https://www.cityjet.com/')

time.sleep(delayed*(1+random.random()))  

## Fill Form Search

In [144]:
from_bar_xpath = '//*[@id="book-flight"]/fieldset[2]/div[1]/div[1]/div/input'
search_bar = browser.find_by_xpath(from_bar_xpath)
search_bar.fill(flight_from)

from_bar_xpath = '//*[@id="book-flight"]/fieldset[2]/div[1]/div[1]/div/div[2]/ul/li'
search_bar = browser.find_by_xpath(from_bar_xpath)
search_bar.click()

time.sleep(delayed*(1+random.random()))  

In [145]:
to_bar_xpath = '//*[@id="book-flight"]/fieldset[2]/div[1]/div[2]/div/input'
search_bar = browser.find_by_xpath(to_bar_xpath)
search_bar.fill(flight_to)

to_bar_xpath = '//*[@id="book-flight"]/fieldset[2]/div[1]/div[2]/div/div[2]/ul/li'
search_bar = browser.find_by_xpath(to_bar_xpath)
search_bar.click()

time.sleep(delayed*(1+random.random()))  

In [146]:
date_from_bar_xpath = '//*[@id="book-flight"]/fieldset[2]/div[2]/div[1]/input'
search_bar = browser.find_by_xpath(date_from_bar_xpath)
search_bar.fill(date_from)

time.sleep(delayed*(1+random.random()))  

In [147]:
date_to_bar_xpath = '//*[@id="book-flight"]/fieldset[2]/div[2]/div[2]/input'
search_bar = browser.find_by_xpath(date_to_bar_xpath)
search_bar.fill(date_to)

time.sleep(delayed*(1+random.random()))  

In [148]:
select_bar_xpath = '//*[@id="book-flight"]/fieldset[4]/div[1]/select/option[1]'
search_bar = browser.find_by_xpath(select_bar_xpath)
search_bar.click()

time.sleep(delayed*(1+random.random()))  

In [149]:
search_button_xpath = '//*[@id="book-flight"]/div[1]/a' 
search_button = browser.find_by_xpath(search_button_xpath)
search_button.click()

time.sleep(delayed*(1+random.random()))  

## Search Page Result

In [150]:
search_results_xpath = '//*[@id="footer-outer"]/div/div[3]/div[2]/div[1]/div[1]/div[1]/ul/li[1]/a' 
search_results = browser.find_by_xpath(search_results_xpath)
print(search_results.value)
print(search_results['href'])

About Us
https://www.cityjet.com/about/


In [153]:
scraped_data = []

with browser.get_iframe('pass-data-iframe') as iframe:

    # for OUTBOUND
    
    xflights = '//*[@id="fpow_avail_tb1"]/tbody/tr' 
    flights = iframe.find_by_xpath(xflights)
    
    xdest = '//*[@id="si_outbound"]/p[1]'
    dest = flights.find_by_xpath(xdest)
    print("[{}]".format(dest.value))

    for flightIdx, flight in enumerate(flights):
        print("Flight: ", flightIdx)
        blocks = flight.find_by_css('td[class="ff_bgrd1"]')
        scraped_detail = ["From " + flight_from, "To " + flight_to, "Date " + date_from]
        for blockIdx, block in enumerate(blocks):
            print("Block1: ", blockIdx)
         
            xdetails = 'td[class="label"]'
            details = block.find_by_css(xdetails)
            for detailIdx, detail in enumerate(details):
                print("Detail1: ", detailIdx, detail.value.replace('\n', ' '))
                scraped_detail.append(detail.value.replace('\n', ' '))
                
        xdetails = 'span[class="nowrap fpow_span_price"]'
        details = flight.find_by_css(xdetails)
        scraped_prices = []
        for detailIdx, detail in enumerate(details):
            print("Detail All Prices: ", detailIdx, detail.value)   
            scraped_prices.append(detail.value)
                
        blocks = flight.find_by_css('td[class="ff_bgrd2 fpow_bgrd2 VALUE"]')
        scraped_price_value = ''
        for blockIdx, block in enumerate(blocks):
            print("Block2: ", blockIdx)
          
            xdetails = 'span[class="nowrap fpow_span_price"]'
            details = block.find_by_css(xdetails)
            for detailIdx, detail in enumerate(details):
                print("Detail CityValue: ", detailIdx, detail.value)    
                scraped_price_value = detail.value
          
        # create function VALUE for other prices ...
        # clean, format, check error on data
        
        if scraped_detail and scraped_prices:
            scraped_data.append((dest.value, scraped_detail, scraped_prices, scraped_price_value)) 
        
                
                

[OUTBOUND: LONDON » AMSTERDAM]
Flight:  0
Flight:  1
Block1:  0
Detail1:  0 Departs 07:45
Detail1:  1 Arrives 10:00
Detail1:  2 Duration 1:15
Detail1:  3 Stop(s) 0
Detail1:  4 Flight WX6082
Detail All Prices:  0 £119.12
Detail All Prices:  1 £201.12
Detail All Prices:  2 £307.12
Block2:  0
Detail CityValue:  0 £119.12
Flight:  2
Block1:  0
Detail1:  0 Departs 09:20
Detail1:  1 Arrives 11:30
Detail1:  2 Duration 1:10
Detail1:  3 Stop(s) 0
Detail1:  4 Flight WX6086
Detail All Prices:  0 £119.12
Detail All Prices:  1 £201.12
Detail All Prices:  2 £307.12
Block2:  0
Detail CityValue:  0 £119.12
Flight:  3
Block1:  0
Detail1:  0 Departs 19:05
Detail1:  1 Arrives 21:15
Detail1:  2 Duration 1:10
Detail1:  3 Stop(s) 0
Detail1:  4 Flight WX6094
Detail All Prices:  0 £264.12
Detail All Prices:  1 
Detail All Prices:  2 £307.12
Block2:  0
Detail CityValue:  0 £264.12
Flight:  4
Block1:  0
Detail1:  0 Departs 20:05
Detail1:  1 Arrives 22:20
Detail1:  2 Duration 1:15
Detail1:  3 Stop(s) 0
Detail1: 

In [154]:
print(scraped_data)

[('OUTBOUND: LONDON » AMSTERDAM', ['From London', 'To Amsterdam', 'Date 12/07/2018', 'Departs 07:45', 'Arrives 10:00', 'Duration 1:15', 'Stop(s) 0', 'Flight WX6082'], ['£119.12', '£201.12', '£307.12'], '£119.12'), ('OUTBOUND: LONDON » AMSTERDAM', ['From London', 'To Amsterdam', 'Date 12/07/2018', 'Departs 09:20', 'Arrives 11:30', 'Duration 1:10', 'Stop(s) 0', 'Flight WX6086'], ['£119.12', '£201.12', '£307.12'], '£119.12'), ('OUTBOUND: LONDON » AMSTERDAM', ['From London', 'To Amsterdam', 'Date 12/07/2018', 'Departs 19:05', 'Arrives 21:15', 'Duration 1:10', 'Stop(s) 0', 'Flight WX6094'], ['£264.12', '', '£307.12'], '£264.12'), ('OUTBOUND: LONDON » AMSTERDAM', ['From London', 'To Amsterdam', 'Date 12/07/2018', 'Departs 20:05', 'Arrives 22:20', 'Duration 1:15', 'Stop(s) 0', 'Flight WX6098'], ['£144.12', '£201.12', '£307.12'], '£144.12')]


In [155]:
df = pd.DataFrame(data=scraped_data, columns=["Title", "Details", "All prices", "CityValue"]) 

if not os.path.isfile(scrapedCsv):
    df.to_csv(scrapedCsv)
else:
    df.to_csv(scrapedCsv, mode='a', index=False, header=False)
    
print(df)

                          Title  \
0  OUTBOUND: LONDON » AMSTERDAM   
1  OUTBOUND: LONDON » AMSTERDAM   
2  OUTBOUND: LONDON » AMSTERDAM   
3  OUTBOUND: LONDON » AMSTERDAM   

                                             Details  \
0  [From London, To Amsterdam, Date 12/07/2018, D...   
1  [From London, To Amsterdam, Date 12/07/2018, D...   
2  [From London, To Amsterdam, Date 12/07/2018, D...   
3  [From London, To Amsterdam, Date 12/07/2018, D...   

                    All prices CityValue  
0  [£119.12, £201.12, £307.12]   £119.12  
1  [£119.12, £201.12, £307.12]   £119.12  
2         [£264.12, , £307.12]   £264.12  
3  [£144.12, £201.12, £307.12]   £144.12  


In [156]:
browser.quit()